In [97]:
from dotenv import load_dotenv

In [98]:
from langchain_community.document_loaders import TextLoader

In [99]:
loader = TextLoader("machine.txt", encoding="utf-8")


In [100]:
docs=loader.load()

In [101]:
len(docs)

1

In [102]:
docs

[Document(metadata={'source': 'machine.txt'}, page_content='Title: Comprehensive Overview of Machine Learning\n\nMachine Learning (ML) is a branch of artificial intelligence (AI) that enables systems to learn from data and improve their performance without explicit programming. It leverages statistical techniques to find patterns, make predictions, and support decision-making across various domains, from healthcare to finance and autonomous systems.\n\nTypes of Machine Learning\n\nSupervised Learning\nSupervised learning trains models on labeled datasets, meaning each training example comes with input-output pairs. Algorithms learn to map inputs to outputs by minimizing prediction errors.\n\nRegression predicts continuous values (e.g., house prices, temperature).\n\nClassification predicts discrete labels (e.g., spam detection, image recognition).\nCommon algorithms: Linear regression, logistic regression, decision trees, support vector machines (SVM), k-nearest neighbors (k-NN), and n

In [103]:
##CHUNKING DATA 

In [104]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [105]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)


In [106]:
chunks

[Document(metadata={'source': 'machine.txt'}, page_content='Title: Comprehensive Overview of Machine Learning'),
 Document(metadata={'source': 'machine.txt'}, page_content='Machine Learning (ML) is a branch of artificial intelligence (AI) that enables systems to learn'),
 Document(metadata={'source': 'machine.txt'}, page_content='systems to learn from data and improve their performance without explicit programming. It leverages'),
 Document(metadata={'source': 'machine.txt'}, page_content='It leverages statistical techniques to find patterns, make predictions, and support decision-making'),
 Document(metadata={'source': 'machine.txt'}, page_content='decision-making across various domains, from healthcare to finance and autonomous systems.'),
 Document(metadata={'source': 'machine.txt'}, page_content='Types of Machine Learning'),
 Document(metadata={'source': 'machine.txt'}, page_content='Supervised Learning'),
 Document(metadata={'source': 'machine.txt'}, page_content='Supervised learn

In [107]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [108]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [109]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [110]:
type(embeddings)

langchain_community.embeddings.huggingface.HuggingFaceEmbeddings

In [111]:
#creating a retriever

In [112]:
# The retriever is used to query your vector store:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [113]:
# Connect a language model (LLM)
# For testing inside code, we can use any model we have access to:

In [114]:
#first load environment variables
load_dotenv()

True

In [115]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

In [116]:
# query='deep'

In [117]:
# result=vectorstore.similarity_search(query,k=3)

In [118]:
# result[0].page_content

In [119]:
##prompt engineering and template creation 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create a good RAG prompt template
template = """You are a helpful, unbiased chatbot. 
Use the provided context to answer the question accurately.
If the answer is not in the context, say you don’t know. 
Do not make up information.

Context:
{context}

Question:
{question}

Answer:"""


prompt = ChatPromptTemplate.from_template(template)

In [120]:
##initialize llm 

In [121]:
model = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

In [122]:
##create chain from langchain.chains import RetrievalQA

In [123]:
rag_chain = (
    {
        "context": retriever,
        "question": lambda x: x #Take the user input x and feed it to both retriever and lambda.
    }
    | prompt
    | model
    | StrOutputParser()
)

In [124]:
response=rag_chain.invoke("Training & Optimization: Use of loss functions, gradient descentand hyperparameter tuning to optimize models")

In [125]:
response

'The provided statement "Training & Optimization: Use of loss functions, gradient descent, and hyperparameter tuning to optimize models" is partially found in the context. \n\nThe first document mentions "Training & Optimization: Use of loss functions, gradient descent, and hyperparameter tuning to", and the second document continues with "tuning to optimize models." \n\nSo, the complete statement is confirmed by combining information from the first two documents.'